### Step 1: Loading Dataset

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
from torchvision.datasets import ImageFolder

In [3]:
from torchvision import transforms

In [4]:
import os
from glob import glob
import numpy as np

In [5]:
transform = transforms.Compose([transforms.Resize((320,320)),
                                transforms.ToTensor(),
                                transforms.Normalize([.5,.5,.5],[.5,.5,.5])
                               ])

train_dataset = ImageFolder("data/train/",transform=transform)
valid_dataset = ImageFolder("data/valid/",transform=transform)

train_dataset.__len__(),valid_dataset.__len__()

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/'

### Step 2: Data Preprocess

In [6]:
from torch.utils.data import  DataLoader

In [7]:
train_dl = DataLoader(train_dataset,batch_size=16,shuffle=True)
valid_dl = DataLoader(valid_dataset,batch_size=16,shuffle=True)

NameError: name 'train_dataset' is not defined

In [9]:
train_gen = iter(train_dl)

In [10]:
next(train_gen)

[
 ( 0 , 0 ,.,.) = 
   0.7882  0.7882  0.7882  ...   0.9294  0.9294  0.9294
   0.7961  0.7961  0.7961  ...   0.9294  0.9294  0.9294
   0.8196  0.8196  0.8196  ...   0.9294  0.9294  0.9294
            ...             ⋱             ...          
   0.9686  0.9686  0.9686  ...   0.9451  0.9451  0.9451
   0.9686  0.9686  0.9686  ...   0.9451  0.9451  0.9451
   0.9451  0.9451  0.9529  ...   0.9451  0.9451  0.9451
 
 ( 0 , 1 ,.,.) = 
   0.1686  0.1686  0.1686  ...  -0.0902 -0.0902 -0.0902
   0.1765  0.1765  0.1765  ...  -0.0902 -0.0902 -0.0902
   0.2000  0.2000  0.2000  ...  -0.0902 -0.0902 -0.0902
            ...             ⋱             ...          
   0.5294  0.5294  0.5294  ...   0.4745  0.4745  0.4745
   0.5294  0.5294  0.5294  ...   0.4745  0.4745  0.4745
   0.5373  0.5373  0.5451  ...   0.4745  0.4745  0.4745
 
 ( 0 , 2 ,.,.) = 
  -0.9608 -0.9608 -0.9608  ...  -0.0118 -0.0118 -0.0118
  -0.9529 -0.9529 -0.9529  ...  -0.0118 -0.0118 -0.0118
  -0.9294 -0.9294 -0.9294  ...  -0.0118 -0.0

### Step 3: Create Model

In [12]:
from torch.nn import functional as F
from torch import nn

In [13]:
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(16, 32, 3,stride=1,padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 3,stride=1,padding=1)
        
        self.conv4 = nn.Conv2d(64, 128, 3,stride=1,padding=1)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128, 256, 3,stride=1,padding=1)
        
        self.conv6 = nn.Conv2d(256, 512, 3,stride=1,padding=1)
        self.conv6_bn = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(512 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 4)

        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4_bn(self.conv4(x))))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6_bn(self.conv6(x))))
        x = x.view(-1, 512 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        
        return x

### Step4: Instantiate Model Class

In [14]:
model = CNNModel()

### Step 5: Instantiate Loss Class
- CNN: Cross Entropy Loss

In [15]:
criterion = nn.CrossEntropyLoss()

### Step 6: Instantiate Optimizer Class

In [16]:
learning_rate =0.01
optimizer =torch.optim.SGD(model.parameters(), lr= learning_rate)

### Parameters Review

In [17]:
print(model.parameters())
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())

<generator object Module.parameters at 0x1180e7728>
22
torch.Size([16, 3, 3, 3])


In [18]:
batch_size = 16
n_iters = 100000
num_epochs = n_iters/(len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
num_epochs

2266

### Step7. Run the model

In [19]:
def accuracy(images,labels):
    #Forward pass only to get logits/ output
    outputs = model(images)
                
                #Get predictions from the maximu value
    _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
    total= labels.size(0)
                
                #Total correct predictions
    correct=(predicted == labels.data).sum()
            
    return  100 * correct/ total

from tqdm import trange

for epoch in range(num_epochs):
    t=trange(len(train_dl))
    train_gen = iter(train_dl)
    loss_m =[]
    acc_m = []
    for i in t:
        #Load imgs as Variable
        images,labels=next(train_gen)

        images = Variable(images)
        labels = Variable(labels)
        
        #Clear gradient w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get output/logits
        outputs = model(images)
        
        #Calculate Loss: softmax - >cross entrophy loss
        loss = criterion(outputs, labels)
        
        #Getting gradients w.r.t. parameters
        loss.backward()
        acc_ = accuracy(images,labels)
        loss_m.append(loss.data[0])
        acc_m.append(acc_)
        #Updating parameters
        optimizer.step()
        
        window=3
        if i % window == window-1:
            t.set_description("i%s loss:%.3f\tacc:%.3f"%(i,sum(loss_m)/window,sum(acc_m)/window))
            loss_m,acc_m = [],[]
            
    acc=0
            # Iterate through test dataset
    for images, labels in valid_dl:
                #Load images to a Torch Variable
        images = Variable(images)
        labels = Variable(labels)
        acc += accuracy(images,labels)
            
    acc = acc/len(valid_dl)
            #print Loss
    print("Iteration:%.3f\t Loss:%.3f\t Acc:%.3f."%(i, loss.data[0], acc))
            

  0%|          | 0/45 [00:00<?, ?it/s]

Iteration:44.000	 Loss:1.150	 Acc:36.250.


  0%|          | 0/45 [00:00<?, ?it/s]

Iteration:44.000	 Loss:0.894	 Acc:53.750.


  0%|          | 0/45 [00:00<?, ?it/s]

...

Iteration:44.000	 Loss:0.746	 Acc:75.000.


  0%|          | 0/45 [00:00<?, ?it/s]

Iteration:44.000	 Loss:0.748	 Acc:82.500.


i2 loss:0.765	acc:97.917:   9%|▉         | 4/45 [00:32<05:30,  8.06s/it]

KeyboardInterrupt: 

In [5]:
cates = ["w","h","non","l"]

In [11]:
train_dataset.class_to_idx

{'h': 0, 'l': 1, 'non': 2, 'w': 3}

In [ ]:
pics = glob("data/train/*/*")

pics

for cate in cates:
    os.system("mkdir -p data/valid/%s"%(cate))

to_valid=np.random.choice(pics,70).tolist()

def replace_train(x):
    return x.replace("train","valid")

to_valid2=list(replace_train(url) for url in to_valid)